# (day4) Section 5: Transformer

本書は、「深層学習後編（day4）レポート」の、「Section 5: Transformer」についてのものです。


## 1. 要点まとめ


<!-- - Seq2seq
- page. 19
  - RNN
  - 言語モデル
  - RNN x 言語モデル
  - Encoder - Decoder
  - 実装 -> "実装演習" で確認 -->


<!-- 
- Transformer - Self-Attention ( 事故注意結合 )
- page. 23
  - ニューラル機械翻訳の問題点
  - Attention
  - Transformer
  - 主要モジュール
    - Attention
      - Source Target Attention
      - Self-Attention
    - Transformer-Encoder
    - Self-Attention
    - Position-Wise Feed-Forward-Networks
    - Scaled dot product attention
    - Multi-head attention
    - Decoder
      - Self-Attention
      - Encoder-Decoder attention
    - Add & Norm
      - Add ( Residual Connection )
      - Norm ( Layer Normarization )
    - Position Encoding
  - 実装 -> "実装演習" で確認 -->


- NOTE: https://arxiv.org/pdf/1409.3215.pdf


### Seq2seq ( Encoder-Decoder モデル )

Transformer を理解するために必要な材料としての、 Seq2seq 。

Encoder-Decoder モデルと呼ばれ、入力系列を Encode して内部状態に変換。
その内部状態を Decode して新たな表現を作成する。

Seq2seq の中の材料として、

- RNN
- 言語モデル


#### RNN ( Recurrent Neural Network )

系列データの、前の時刻の出力を現在の入力に、再帰的に計算。

系列データの全て ( 例えば、一文 ) を、1つの内部状態ベクトルで表現。


#### 言語モデル

単語の並び ( 1, ... m-1, m ) を、事後確率で表現。

$$
P(w_1, \cdots, w_m)
=\prod_{i=1}~m
P(w_i \mid w_1, \cdots, w_{i-1})
$$

それぞれの単語の同時確率を求める。

$$
argmax_{w \in V} P(w_1, \cdots, w_v)
$$

( V は全ての単語 )

RNN を使用して、各時点で度の単語を使えば自然 ( 事後確率最大 ) かを出力できる。


#### Encoder - Decoder

##### Encoder

Source から Embed 表現にした単語の入力を Encoder RNN で学習する。

Encoder の学習方法結果が、内部状態ベクトルとして出力される。

##### Decoder

Decoder RNN は、Encoder の出力の、内部状態ベクトルから、
新しい表現の各単語の事後確率を出力する。
( Softmax 活性化関数で、各単語の確率を計算する )

##### 学習

Decoder の出力と、訓練データの正解から誤差関数を使用すれば、
教師あり学習として、 Encoder から Decoder まで End2end で学習できる。



### Transformer - Self-Attention ( 事故注意機構 )

#### ニューラル機械翻訳の問題点

翻訳元の文を 1 つのベクトルで表現しているため、文長が長くなると表現力が不足する。


#### Attention

文長が長くなると表現力が不足する問題点を解決するために、翻訳先の各単語を選択するときに、翻訳元の文中の各単語の隠れ状態 ( ベクトル ) を利用する。

<!-- - TODO: 隠れ状態? -->

翻訳元の文中の各単語の隠れ状態の、加重平均を、

$$
c_i = \sum_{j=1}^{T_{x}} \alpha_{ij} h_j \cdot
$$

重み ( 全て足すと 1 ) は、 FFNN で求める。

$$
\alpha_{ij} = \frac{\exp(e_{ij})}
{\sum_{k=1}^{T_x} \exp(e_{ik})}, \\
e_{ij} = a(s_{i-1}, h_j)
$$


Attention は、key を索引にした辞書オブジェクトの様に、 query で key に一致する value の、 Attention weight value を取り出すことができる。

<!-- - TODO: 図の Attention weight value?</br>
  valuet が、 Attention weight value のこと? -->

- NOTE: [Multi-head attention mechanism: "queries", "keys", and "values," over and over again - Data Science Blog](https://data-science-blog.com/blog/2021/04/07/multi-head-attention-mechanism/)<br/>
  > Please remember this mantra of attention mechanism: “you compare the ‘query’ with the ‘keys’ and get scores/weights for the ‘values.’

#### Transformer

RNN を使わずに Attention を使用して、少ない計算量を実現。

#### 主要モジュール

以降に登場する主要モジュールは、以下の通り。

- Attention
  - Source Target Attention
  - Self-Attention
- Transformer-Encoder
- Self-Attention
- Position-Wise Feed-Forward-Networks
- Scaled dot product attention
- Multi-head attention
- Decoder
  - Self-Attention
  - Encoder-Decoder attention
- Add & Norm
  - Add ( Residual Connection )
  - Norm ( Layer Normarization )
- Position Encoding


#### Attention

注意機構 ( Attention ) の種類は以下の 2 つ。

- Source Target Attention

$$
\sigma(Query=Target, Key=Source) Value=Source
$$

- Self-Attention

$$
\sigma(Query=Source, Key=Source) Value=Source
$$


#### Transformer-Encoder

6層と、自己注意機構 ( Attention ) と Feed Forward NN から成る。


#### Self-Attention

例として、 $ i = 1, 2, 3 $

1. 入力 $ x_i $ と 重み $ W^k $ から、 key $ k_i $ 、<br/>
  入力 $ x_i $ と 重み $ W^v $ から、 key $ v_i $ を得る。

2. query と key の内積 $ q_i \cdot k_i $ に、
  softmax 関数を適用して、確率を求める。<br/>
  ( ここではこれを、 $ p_i $ とする )

3. Feed Forward Network で、

$$
z_1 = \sum_{i=1}^3 p_i v_i \\
output = FFN(z_1)
$$


#### Position-Wise Feed-Forward-Networks

Attention 層の出力を決定する、 2 備全結合 NN。

線形変換 ( $ W_1 $ ) -> ReLu -> 線形変換 ( $ W_2 $ )

$$
FFN(\textbf{x}) = \max(0, \textbf{x} W_1 + \textbf{b}_1) W_2 + \textbf{b}_2
$$


#### Scaled dot product attention

全単語に関する Attention をまとめて計算。

$$
Attention(\textbf{Q}, \textbf{K}, \textbf{V})
=
softmax(
  \frac{\textbf{Q} \textbf{K}^T}
  {
    \sqrt{d_k}
  } \textbf{V}
)
$$


#### Multi-head attention

重みパラメタの異なる８個の Scaled dot product attention ( ヘッド ) の出力を連結。

```python
        # 各ヘッドの結果を連結(p16左側`Concat`)
        # torch.splitでbatch_sizeごとのn_head個のテンソルに分割
        outputs = torch.split(outputs, batch_size, dim=0)  # (batch_size, len_q, d_model) * n_head
        # dim=-1で連結
        outputs = torch.cat(outputs, dim=-1)  # (batch_size, len_q, d_model*n_head)
```


#### Decoder

Encoder 同様、 6 層。


##### Self-Attention

未来の情報を見ないようにマスクしている。


##### Encoder-Decoder attention

Encoder の出力からの attention を Decoder の Multi-Head Attention で受ける。

<!-- - TODO: 図のどこのことか? -->


#### Add & Norm


##### Add ( Residual Connection )

入出力の差分を学習して、学習・テストエラーを低減させる。

```python
        # residual connectionのための入力 出力に入力をそのまま加算する
        residual = q
```


##### Norm ( Layer Normarization )

学習の高速化のための、正則化。

- NOTE: (day3) Section 3: 軽量化 - 高速化技術


#### Position Encoding

RNN を使用しない代わりの、単語列語順情報。

$$
PE_{(pos, 2i)} = \sin{(
  \frac{pos}
  {10000^{2i / 512}}
)} \\
PE_{(pos, 2i + 1)} = \cos{(
  \frac{pos}
  {10000^{2i / 512}}
)}
$$

- pos: 単語の位置
- i: 成分の次元
- 512: モデルの次元
